In [1]:
import os
from supabase import create_client, Client
import requests
from dotenv import load_dotenv
from koboextractor import KoboExtractor

In [2]:
# Loading environment variables
load_dotenv()
KOBO_API_KEY = os.getenv("KOBO_API_KEY")
KOBO_FORM_ID = os.getenv("KOBO_FORM_ID")
KOBO_BASE_URL = 'https://kf.kobotoolbox.org/api/v2'
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_API_KEY = os.getenv("SUPABASE_API_KEY")

In [3]:
# Connect to Kobo Account using my token 
kobo = KoboExtractor(KOBO_API_KEY, KOBO_BASE_URL)

In [8]:
# Fetch data from a specific form
data = kobo.get_data(KOBO_FORM_ID)
data

{'count': 13,
 'next': None,
 'previous': None,
 'results': [{'_id': 539546599,
   'formhub/uuid': 'b83aa11749194b35b50090f6b5fb1f41',
   'start': '2025-08-15T09:48:34.089+07:00',
   'end': '2025-08-15T09:49:46.890+07:00',
   'Demografi/Usia_tahun': '44',
   'Demografi/Pendidikan_terakhir': 'sma',
   'Demografi/Jenis_kelamin': 'laki_laki',
   'Demografi/Kerja_utama': 'umkm',
   'Demografi/Kelompok_mana_diikuti': 'umkm',
   'UMKM/Jenis_produk_utama': 'terasi',
   'UMKM/Kuantitas_terjual_1bulan': '9000',
   'UMKM/Pendapatan_bulan_dalam_Rupiah_001': '18000000',
   '__version__': 'v5PEgRCv5H6eMgqqcPW2aT',
   'meta/instanceID': 'uuid:dcd966a8-8523-46b6-a526-0e2dc773daec',
   '_xform_id_string': 'aRQ28soDv3f4PxbHBNgRzn',
   '_uuid': 'dcd966a8-8523-46b6-a526-0e2dc773daec',
   'meta/rootUuid': 'uuid:dcd966a8-8523-46b6-a526-0e2dc773daec',
   '_attachments': [],
   '_status': 'submitted_via_web',
   '_geolocation': [None, None],
   '_submission_time': '2025-08-15T02:49:47',
   '_tags': [],
   '_

The following cell contains mappings for all of the fields in the urvey as of September 2, 2025. In future revisions of the survey, the following cell might need to be modified so as to ensure correct imports.

In [48]:
# Define field mappings
field_mappings = {
    '_uuid': '_uuid',
    'time_of_response': '_submission_time',
    '__version_id': '__version__',
    'pendidikan_terakhir': 'Demografi/Pendidikan_terakhir',
    'jenis_kelamin': 'Demografi/Jenis_kelamin',
    'kerja_utama': 'Demografi/Kerja_utama',
    'kelompok_mana': 'Demografi/Kelompok_mana_diikuti',
    'nama': 'Demografi/Nama_lengkap',
    'perbaikan_survey': 'Demografi/Apakah_Anda_memiliki_perbaikan_survei_ini',

    # These following values might need to be converted int.
    'usia': 'Demografi/Usia_tahun',
    'lama_tinggal': 'Demografi/Lama_tinggal',
}

# Define mappings for fields with string values
text_fields = [

    'Nelayan/hasil_tangkapan_utama',
    'UMKM/Jenis_modal',
    'UMKM/Tempat_menjual',    
    'UMKM/Jenis_produk_utama',
    'UMKM/Apa_produk_Anda_halal',
    'Petambak/Jenis_budidaya_utama',
    'Petambak/Jenis_lain_panen_di_tambak',
    'Petani_Sawah/Sawah_dekat_mangrove',
    'Petani_Sawah/Pernah_intrusi',
    'Petani_Sawah/Kondisi_membaik_dalam_2_tahun',
    'Petani_Sawah/Menurut_Anda_apakah_tau_intrusi_air_laut',
    'Petani_Sawah/Sejak_adanya_rehabil_sil_panen_sawah_Anda',
    'KTH/Jenis_kegiatan'

]

# Define mappings for fields with integer/float values
numeric_fields = [

    'Nelayan/Jumlah_hasil_ikan',
    'Nelayan/Harga_jual_ikan',
    'Nelayan/Jumlah_hasil_udang',
    'Nelayan/Harga_jual_udang',
    'Nelayan/Jumlah_hasil_kepiting',
    'Nelayan/Harga_jual_kepiting',
    'Nelayan/Jumlah_hasil_kerang',
    'Nelayan/Harga_jual_kerang',
    'Nelayan/Pendapatan_bulan_dalam_Rupiah',

    'UMKM/Harga_jualan_produk',
    'UMKM/Pendapatan_bulan_dalam_Rupiah_001',

    'Petambak/Jumlah_udang_vaname',
    'Petambak/Harga_udang_vaname',
    'Petambak/Jumlah_udang_windu',
    'Petambak/Harga_udang_windu',
    'Petambak/Berapa_minggu_satu_musin',

    'KTH/Berapa_kali_KTH_kegiatan',

    'Sadar_Wisata/Wisata_per_musin'

]

fields_with_zero_value = [
    'UMKM/Kuantitas_modal',
    'UMKM/Kuantitas_terjual_1bulan',
    'UMKM/Berapa_total_biaya_p_silkan_dalam_rupiah',
]

In [49]:
def transform_kobo_record(record):
    """Transform a single Kobo record into database format"""
    
    # Extract mandatory fields with defaults
    result = {}
    for db_field, kobo_field in field_mappings.items():
        result[db_field] = record.get(kobo_field, None)

    # Extract flexible fields
    kobo_data = {}
    all_mapped_fields = set(text_fields + numeric_fields)
    for key, value in record.items():
        if key in fields_with_zero_value:
            kobo_data[key] = float(value)
        elif key in all_mapped_fields and value not in [None, "", "AUTOMATIC", "0"]:
            if key in numeric_fields:
                kobo_data[key] = float(value)
            else:
                kobo_data[key] = value
    
    result['kobo_data'] = kobo_data

    return result
    


In [50]:
transformed_data = [transform_kobo_record(record) for record in data['results']]
transformed_data

[{'_uuid': 'dcd966a8-8523-46b6-a526-0e2dc773daec',
  'time_of_response': '2025-08-15T02:49:47',
  '__version_id': 'v5PEgRCv5H6eMgqqcPW2aT',
  'pendidikan_terakhir': 'sma',
  'jenis_kelamin': 'laki_laki',
  'kerja_utama': 'umkm',
  'kelompok_mana': 'umkm',
  'nama': None,
  'perbaikan_survey': None,
  'usia': '44',
  'lama_tinggal': None,
  'kobo_data': {'UMKM/Jenis_produk_utama': 'terasi',
   'UMKM/Kuantitas_terjual_1bulan': 9000.0,
   'UMKM/Pendapatan_bulan_dalam_Rupiah_001': 18000000.0}},
 {'_uuid': '7cf99d85-c340-461a-99fc-b5b995fe74e0',
  'time_of_response': '2025-08-15T02:51:29',
  '__version_id': 'v5PEgRCv5H6eMgqqcPW2aT',
  'pendidikan_terakhir': 'sd',
  'jenis_kelamin': 'laki_laki',
  'kerja_utama': 'Nelayan',
  'kelompok_mana': 'nelayan',
  'nama': None,
  'perbaikan_survey': None,
  'usia': '44',
  'lama_tinggal': '3',
  'kobo_data': {'Nelayan/hasil_tangkapan_utama': 'udang rebon',
   'Nelayan/Jumlah_hasil_udang': 300.0,
   'Nelayan/Pendapatan_bulan_dalam_Rupiah': 4500000.0}},

Convert from a nested .JSON into a file format that is amenable for SQL posting
Given that:
1. Certain questions are not mandatory, and thus will not show up in some answers
2. Of the mandatory fields, some are null because in the first version we didn't add the questions

In [ ]:
# Convert from a nested .JSON into a file format that is amenable for SQL posting
# Given that certain questions are not mandatory, and thus will not show up in some answers
# Of the mandatory fields, some are null because in the first version we didn't add the questions
value = {
    '_uuid' : ,
    'time_of_response': ,
    '__version_id': ,
    'usia': ,
    'pendidikan_terakhir': ,
    'nama': ,
    'jenis_kelamin': ,
    'lama_tinggal': ,
    'kerja_utama': ,
    'kelompok_mana': ,
    'kobo_data': {
        # The data inside here will be flexible, 
        # as certain questions are optional, 
        # and thus don't show up in every export
    }
}

In [ ]:
# Fetch data that already exists in the database

In [ ]:
# For each dictionary entry in the insert object, compare to the data that already exists. If it already exists, skip this line

In [4]:
# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_API_KEY)